In [ ]:
import os
import base64
from openai import OpenAI
from tqdm import trange

In [ ]:
# Initialize the OpenAI client
client = OpenAI(api_key="api-key")

In [ ]:
# -------------------------------
# Configuration
# -------------------------------
SOURCE_DIR = "Requirements"          # Folder containing .txt requirement files
DEST_DIR = "output_folder"         # Folder to save generated UML diagram images
MODEL_NAME = "gpt-image-1"             # Model used for image generation

# Create destination folder if not exists
os.makedirs(DEST_DIR, exist_ok=True)

In [ ]:
# -------------------------------
# Prompt Template
# -------------------------------
def generate_prompt(requirement_text):
    return f"""
You are a Software Architect AI specializing in UML modeling and architectural design.
Analyze the following software requirements and synthesize a UML Component Diagram that visually represents the system architecture.
Your goals:
1. Identify key software components, subcomponents, and external systems.
2. Show all major interfaces (lollipop/socket) and dependencies (dashed arrows).
3. Organize the diagram following an appropriate architectural style inferred from the requirements.
4. Maintain a professional UML style — white background, clean labels, rectangular components with tabs.

Output:
Return only a high-resolution UML component diagram image.

Software requirements:
{requirement_text}
"""

# ---------------------------

In [ ]:
# Gather all text files
# -------------------------------
text_files = [f for f in os.listdir(SOURCE_DIR) if f.endswith(".txt")]
total_files = len(text_files)

if total_files == 0:
    print("No text files found in dataset folder.")
    exit()

In [ ]:
# -------------------------------
# Main Processing Loop
# -------------------------------
for i in trange(total_files, desc="Generating UML Diagrams"):
    filename = text_files[i]
    input_path = os.path.join(SOURCE_DIR, filename)
    output_path = os.path.join(DEST_DIR, os.path.splitext(filename)[0] + ".png")

    # Skip if already generated
    if os.path.exists(output_path):
        continue

    try:
        # Read requirement
        with open(input_path, "r", encoding="utf-8") as f:
            requirement_text = f.read().strip()

        # Skip empty files
        if not requirement_text:
            print(f"Skipping empty file: {filename}")
            continue

        # Create prompt
        prompt = generate_prompt(requirement_text)

        # Generate UML image
        result = client.images.generate(
            model=MODEL_NAME,
            prompt=prompt,
            size="auto"
        )

        # Decode and save image
        image_base64 = result.data[0].b64_json
        image_bytes = base64.b64decode(image_base64)

        with open(output_path, "wb") as img_file:
            img_file.write(image_bytes)

    except Exception as e:
        print(f"\n Error generating for {filename}: {e}")
        continue

print("\n All UML diagrams processed successfully.")
